In [37]:
import numpy as np
import pandas as pd
import os
from collections import defaultdict
# from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
from utils import *

In [38]:
def hex_to_bin(hex_num):
    
    binary_value = bin(int(str(hex_num), 16))[2:]
    
    return binary_value

def int_to_bin(int_num):
    
    binary_value = bin(int_num)[2:]
    
    return binary_value

hex_to_dec = lambda x: int(x, 16)
dec_to_hex = lambda x : hex(int(x))[2:]

In [39]:
data_folder = '../data/Car Hacking Dataset/'
# data_folder = ''

In [40]:
def read_data(data_path):
    
    columns = ['Timestamp','ID', 'DLC', 'Payload', 'label']
    
    data = pd.read_csv(data_path)
    
    ##Replacing all NaNs with '00' 
    data = data.replace(np.NaN, '00')

    data['ID'] = data['ID'].apply(hex_to_dec)
    
    data['Payload'] = data['Payload'].str.replace(' ', '')
    data['Payload'] = data['Payload'].apply(hex_to_dec)
    
    data = data.assign(IAT=data['Timestamp'].diff().fillna(0))
    
    return data

In [41]:
## Function to calculate average time and frequency of each message ID
## In this function frequency is defined as the number of packets a particular message ID has
## Average time is defined as the average time between two packets of the same message ID



In [42]:
normal_data = read_data(os.path.join(data_folder, 'benign_data.csv'))
normal_data = normal_data[:100_000]

In [43]:
def calculate_average_time_frequency(dataframe):
    
    frame_times = defaultdict(list)
    total_frames = len(dataframe)
    id_frequency = dataframe["ID"].value_counts()
    id_frequency = defaultdict(int, id_frequency)

    for index, row in dataframe.iterrows():
        id_value = row["ID"]
        timestamp_value = row["Timestamp"]
        frame_times[id_value].append(timestamp_value)


    average_times = {}
    for arbitration_id, timestamps in frame_times.items():
        if len(timestamps) > 1:
            differences = [(timestamps[i+1] - timestamps[i]) for i in range(len(timestamps)-1)]
            average_times[arbitration_id] = sum(differences) / len(differences)
        else:
            average_times[arbitration_id] = 0

    return average_times, id_frequency, total_frames

In [44]:
def calculate_periodicity(df):
    # Initialize an empty dictionary to store the periodicity for each can_id
    periodicity_dict = {}

    # Group the DataFrame by 'can_id'
    grouped = df.groupby('ID')

    for can_id, group_df in grouped:
        # Sort the DataFrame by 'timestamp'
        sorted_df = group_df.sort_values(by='Timestamp')

        # Calculate the periodicity for the current can_id
        first_timestamp = sorted_df.iloc[0]['Timestamp']
        last_timestamp = sorted_df.iloc[-1]['Timestamp']
        num_occurrences = len(sorted_df)

        if num_occurrences > 1:
            periodicity = (last_timestamp - first_timestamp) / (num_occurrences - 1)
        else:
            periodicity = 0  # Avoid division by zero if there's only one occurrence

        # Store the periodicity in the dictionary
        periodicity_dict[can_id] = periodicity

    return periodicity_dict

def calculate_inverse_periodicity(periodicity_dict):
    
    inverse_periodicity_dict = {id: 1.0 / periodicity for id, periodicity in periodicity_dict.items()}
    
    return inverse_periodicity_dict

In [45]:
def injection_possible(inter_arrival_time):

    if inter_arrival_time < 0:
        return (False, 0)

    #This is the time that it would take a fully stuffed frame to be transmitted 
    #to the bus at 500kbps 
    time_to_transit = 0.000216

    num_injections = (inter_arrival_time - time_to_transit) // time_to_transit

    if num_injections >= 1:
        return (True, num_injections)
    else:
        return (False, 0)

In [46]:
def calculate_atr(data, ts, periodicity_dict):

    atr_dict = dict()

    """data : The dataflow until the time at which decision to inject packet is taken"""

    ids = list(periodicity_dict.keys())

    last_occurrences = data.drop_duplicates(subset='ID', keep='last')

    for id in ids:

        last_occurrence = last_occurrences[last_occurrences['ID'] == id]['Timestamp']
        
        atr = (ts - last_occurrence)/periodicity_dict[id]

        if len(atr.values) == 0:
            atr = 0.0
        else:
            atr = atr.item()
    
        atr_dict[id] = atr

        atr_dict = dict(sorted(atr_dict.items(), key=lambda item: item[1]))

            
    return atr_dict

In [47]:
def apply_function_to_keys_and_sort(dictionary):
    # Apply the function to keys
    transformed_dict = {key: value * 1e6 for key, value in dictionary.items()}
    
    # Sort the dictionary by values and return a new dictionary
    sorted_dict = {k: v for k, v in sorted(transformed_dict.items(), key=lambda item: item[1])}
    
    return sorted_dict

In [48]:
periodicity_dict = calculate_periodicity(normal_data)
hex_periodicity_dict = apply_function_to_keys_and_sort(periodicity_dict)
# hex_periodicity_dict

In [49]:
list(hex_periodicity_dict.keys())[:17]

[704,
 790,
 809,
 608,
 672,
 399,
 1349,
 848,
 320,
 305,
 304,
 2,
 339,
 880,
 1087,
 1088,
 688]

In [50]:
hex_periodicity_dict

{704: 9991.078213947576,
 790: 9991.27592691561,
 809: 9991.410139130383,
 608: 9991.455775935476,
 672: 9991.455775935476,
 399: 9991.597664065477,
 1349: 9991.686573073233,
 848: 9996.778189507559,
 320: 9999.16617094225,
 305: 9999.166776183712,
 304: 9999.168126337743,
 2: 9999.174178752366,
 339: 9999.265383985028,
 880: 10000.955685973167,
 1087: 10001.05096027255,
 1088: 10001.13812338726,
 688: 10007.510947659204,
 497: 19998.712092638016,
 1201: 19998.716469854116,
 1072: 19998.72699379921,
 1264: 19999.978504128732,
 1520: 50001.74779696199,
 161: 99916.08627155102,
 160: 99916.1193982029,
 1680: 100002.53423552691,
 1442: 999109.4692080629,
 1440: 999111.7458717496}

In [51]:
normal_data[normal_data['DLC'] == 2]['ID'].unique()

array([1520])

In [52]:
def hex_to_bin(hex_value, length):
    return bin(int(hex_value, 16))[2:].zfill(length)    

def generate_binary_dict(periodicity_dict):
    # First, convert keys to hex and then to binary
    hex_dict = {key: dec_to_hex(key) for key in periodicity_dict.keys()}
    
    # hex_dict = periodicity_dict

    # Determine the maximum length of binary strings for consistent padding
    max_bin_length = max(len(bin(int(hex_val, 16))[2:]) for hex_val in hex_dict.values())
    
    # Create binary_dict with padded binary values
    binary_dict = {key: hex_to_bin(hex_val, max_bin_length) for key, hex_val in hex_dict.items()}
    
    # Sort binary_dict by binary values
    sorted_binary_dict = {k: v for k, v in sorted(binary_dict.items(), key=lambda item: item[1])}

    hex_dict = {hex(key)[2:]: value for key, value in sorted_binary_dict.items()}
    
    return sorted_binary_dict

# Example usage
binary_dict = generate_binary_dict(periodicity_dict)
binary_dict

{2: '00000000010',
 160: '00010100000',
 161: '00010100001',
 304: '00100110000',
 305: '00100110001',
 320: '00101000000',
 339: '00101010011',
 399: '00110001111',
 497: '00111110001',
 608: '01001100000',
 672: '01010100000',
 688: '01010110000',
 704: '01011000000',
 790: '01100010110',
 809: '01100101001',
 848: '01101010000',
 880: '01101110000',
 1072: '10000110000',
 1087: '10000111111',
 1088: '10001000000',
 1201: '10010110001',
 1264: '10011110000',
 1349: '10101000101',
 1440: '10110100000',
 1442: '10110100010',
 1520: '10111110000',
 1680: '11010010000'}

In [53]:
arb_priorities = {key: idx for idx, key in enumerate(binary_dict.keys())}
arb_priorities

{2: 0,
 160: 1,
 161: 2,
 304: 3,
 305: 4,
 320: 5,
 339: 6,
 399: 7,
 497: 8,
 608: 9,
 672: 10,
 688: 11,
 704: 12,
 790: 13,
 809: 14,
 848: 15,
 880: 16,
 1072: 17,
 1087: 18,
 1088: 19,
 1201: 20,
 1264: 21,
 1349: 22,
 1440: 23,
 1442: 24,
 1520: 25,
 1680: 26}

In [54]:
def sort_dict_by_value(original_dict):
    # Use the sorted function to sort the dictionary by values
    sorted_dict = dict(sorted(original_dict.items(), key=lambda item: item[1]))
    return sorted_dict

sorted_periodicity_dict = sort_dict_by_value(periodicity_dict)
sorted_periodicity_dict

{704: 0.009991078213947575,
 790: 0.009991275926915611,
 809: 0.009991410139130383,
 608: 0.009991455775935475,
 672: 0.009991455775935475,
 399: 0.009991597664065478,
 1349: 0.009991686573073233,
 848: 0.009996778189507559,
 320: 0.00999916617094225,
 305: 0.009999166776183713,
 304: 0.009999168126337743,
 2: 0.009999174178752367,
 339: 0.009999265383985028,
 880: 0.010000955685973168,
 1087: 0.01000105096027255,
 1088: 0.01000113812338726,
 688: 0.010007510947659204,
 497: 0.019998712092638017,
 1201: 0.019998716469854117,
 1072: 0.01999872699379921,
 1264: 0.019999978504128733,
 1520: 0.05000174779696199,
 161: 0.09991608627155102,
 160: 0.09991611939820291,
 1680: 0.10000253423552691,
 1442: 0.9991094692080629,
 1440: 0.9991117458717496}

In [55]:
hex_keys = list(hex_periodicity_dict.keys())
binary_keys = list(binary_dict.keys())

# Create the scores dictionary
scores = {}

for key in hex_keys:
    # Get the index positions in both dictionaries
    hex_index = hex_keys.index(key)
    binary_index = binary_keys.index(key)
    
    # Sum the index positions and store in the scores dictionary
    scores[key] = hex_index + binary_index

# Sort the scores dictionary based on values
sorted_scores = dict(sorted(scores.items(), key=lambda item: item[1]))

sorted_scores

{2: 11,
 704: 12,
 608: 12,
 399: 12,
 320: 13,
 305: 13,
 304: 13,
 790: 14,
 672: 14,
 809: 16,
 339: 18,
 848: 22,
 161: 24,
 160: 24,
 497: 25,
 688: 27,
 1349: 28,
 880: 29,
 1087: 32,
 1088: 34,
 1072: 36,
 1201: 38,
 1264: 41,
 1520: 46,
 1442: 49,
 1440: 49,
 1680: 50}

In [56]:
ids_to_use = list(sorted_scores.keys())[:10]

def int_to_bin(n, length = 16):
    """Convert an integer to a binary string."""
    return bin(n)[2:].zfill(length)

In [57]:
def arbitration(bin1, bin2):

    """
    bin1 (int): ID1 in binary format
    bin2 (int): ID2 in binary format

    returns True if 1st ID wins arbitration, else False
    """
    
    # Count leading zeros before the first '1' in bin1
    leading_zeros_bin1 = len(bin1) - len(bin1.lstrip('0'))
    # Count leading zeros before the first '1' in bin2
    leading_zeros_bin2 = len(bin2) - len(bin2.lstrip('0'))
    
    # The binary string with more leading zeros wins
    return leading_zeros_bin1 > leading_zeros_bin2


In [58]:
def choose_injection_id(data_flow, next_packet, curr_ts, ids_to_use=ids_to_use):
    
    next_packet_id = next_packet['ID']

    periodicity_dict = calculate_periodicity(data_flow)
    
    atr_dict =  calculate_atr(data_flow, curr_ts, periodicity_dict)

    for id in list(atr_dict.keys()):

        if id in ids_to_use:
            if arbitration(int_to_bin(data_flow[-1][1]), int_to_bin(next_packet[1])):
                return id
        else:
            continue
    
    return ids_to_use[0]

In [59]:
def find_key_with_highest_value(original_dict, id_list):
    # Initialize variables to keep track of the highest value and corresponding key
    max_key = None
    max_value = float('-inf')  # Smallest possible value to start comparison

    # Iterate over the list of IDs
    for key in id_list:
        # Check if the key exists in the dictionary
        if key in original_dict:
            # Update max_key and max_value if the current value is higher
            if original_dict[key] > max_value:
                max_value = original_dict[key]
                max_key = key

    return max_key

In [60]:
# normal_data.drop(['IAT'], axis = 1, inplace=True)

In [61]:
def attack(data, thresh):
    
    last_appended = 0
    data = data.drop(columns=['IAT'],axis = 1)
    standby_packets = int(0.1 * len(data))
    out = []

    injection_count = 0
    ptr = standby_packets
    median_iat = normal_data[:standby_packets].groupby('ID')['IAT'].median().reset_index()

    for ind in tqdm(range(len(data))):
        
        if ind < standby_packets:
            out.append(list(data.iloc[ind].values))
            # print("stand",ind)
            last_appended = ind
        
        else:
            curr_ts = data['Timestamp'][ind]
            prev_ts = data['Timestamp'][ind - 1]

            curr_iat = curr_ts - prev_ts

            possible, num_injections = injection_possible(curr_iat)

            if possible:
                # print("Possible")
            
                periodicity_dict = calculate_periodicity(data[:ind+1])

                atr_dict =  calculate_atr(data[:ind+1], curr_ts, periodicity_dict)
            
                attack_id = max(atr_dict, key=atr_dict.get)

                rand_data = data[:ind][data[:ind]['ID'] == attack_id].sample(1)
                dlc = rand_data['DLC'].item()
                payload =  rand_data['Payload'].item()
                
                frame_length = frame_len(dec_to_hex(attack_id), dlc, dec_to_hex(payload))
                tt = transmission_time(frame_length)


                try:
                    min_thresh = thresh * periodicity_dict[attack_id]
                except TypeError:
                    print(thresh, type(thresh))
                    # print(periodicity_dict[attack_id], type(periodicity_dict[attack_id]))

                last_encountered_ts = data[:ind][data[:ind]['ID'] == attack_id].iloc[-1]['Timestamp']

                lb1 = last_encountered_ts + min_thresh
                # lb2 = prev_ts + 0.000255
                lb2 = curr_ts - tt

                if lb1>lb2:
                        attack_ts = lb1
                else:
                    attack_ts = lb2


                if (attack_ts > prev_ts) and (attack_ts < curr_ts):
                    
                    # print("Possible 2")

                    prev_frame_len = frame_len(dec_to_hex(data.iloc[ind - 1]["ID"].item()),
                                               int(data.iloc[ind - 1]["DLC"].item()),
                                               dec_to_hex(int(data.iloc[ind - 1]["Payload"].item())))


                    prev_frame_tt = transmission_time(prev_frame_len)
            
                    max_delay = attack_ts - (data.iloc[ind-1]['Timestamp'].item() + prev_frame_tt)
                    # print(max_delay)

                    delay = 0
                      
                    for j in range(ptr, ind):

                        # if j != ptr:
                        out.append([data.iloc[j]['Timestamp'].item() + delay, 
                                data.iloc[j]['ID'], data.iloc[j]['DLC'], 
                                data.iloc[j]["Payload"], 0])
                        # print("j!=ptr",j)
                        # print(ptr, ind-1)
                        if j == last_appended + 1:
                            last_appended = j
                        else:
                            # print(last_appended, j)
                            # print("SKIPPED AT J!= PTR")
                            last_appended = j
                        
                        
                        next_id = data.iloc[j+1]['ID'].item()
                        id_priority = arb_priorities[next_id]

                        possible_attack_ids = [key for key, value in arb_priorities.items() if value <= id_priority]

                        if len(possible_attack_ids) > 0:

                            # print("Possible 3")

                            aux_attack_id = find_key_with_highest_value(calculate_atr(data.iloc[:j+1], 
                                                                                      data.iloc[j]['Timestamp'] + delay, 
                                                                                      calculate_periodicity(data.iloc[:j+1])), possible_attack_ids)

                            aux_rand_data = data.iloc[:j][data.iloc[:j]['ID'] == aux_attack_id].sample(1)
                            aux_dlc = aux_rand_data['DLC'].item()
                            aux_payload =  aux_rand_data['Payload'].item()
                            aux_last_encountered_ts = data.iloc[:j][data.iloc[:j]['ID'] == aux_attack_id].iloc[-1]['Timestamp']
                            
                            # if data.iloc[j]['Timestamp'] > aux_last_encountered_ts and (aux_last_encountered_ts + periodicity_dict[aux_attack_id] < data.iloc[j+1]['Timestamp']):
                            #     # aux_attack_ts = data.iloc[j]['Timestamp'] + periodicity_dict[aux_attack_id] 
                            #     aux_attack_ts = aux_last_encountered_ts + periodicity_dict[aux_attack_id] 
                            # else:

                            aux_attack_ts = data.iloc[j]["Timestamp"] + transmission_time(frame_len(dec_to_hex(data.iloc[j]["ID"]), 
                                                                                                        int(data.iloc[j]["DLC"]), 
                                                                                                        dec_to_hex(data.iloc[j]["Payload"])))
                            
                            aux_attack_frame = [aux_attack_ts + delay, aux_attack_id, aux_dlc, aux_payload, 2]
                        
                            # print(aux_attack_ts, data.iloc[j]['Timestamp'],  transmission_time(frame_len(dec_to_hex(data.iloc[j]["ID"]), 
                            #                                                                             int(data.iloc[j]["DLC"]), 
                            #                                                                             dec_to_hex(data.iloc[j]["Payload"]))))

                            if aux_attack_ts + delay > data.iloc[j]['Timestamp']:

                                # print("Possible 4")

                                temp_delay = delay + transmission_time(frame_len(dec_to_hex(aux_attack_id), int(aux_dlc), dec_to_hex(aux_payload)))
                                
                                # delay += transmission_time(frame_len(dec_to_hex(aux_attack_id), int(aux_dlc), dec_to_hex(aux_payload)))

                                if (temp_delay <= max_delay):
                                    # print(delay, max_delay)
                                    # print("Possible 5")
                                    delay += transmission_time(frame_len(dec_to_hex(aux_attack_id), int(aux_dlc), dec_to_hex(aux_payload)))
                                    out.append(aux_attack_frame)
                                    # pass
                                else:             
                                    
     

                                    # aux_attack_id = 


                                    pass                                             

                    ##Iterate through the packets
                    ##Choose attack ID using ATR table with priority higher than next packet
                    # OR
                    # Select some packets in particular to shift and only try to inject in those spots
                    # maintain a variable called delay which will track running delay of packets
                    # Figure out how packets will be appeneded if condition is not met
                    # Add limit of injections                        

                    rand_data = data[:ind][data[:ind]['ID'] == attack_id].sample(1)
                    # print(delay, max_delay, delay < max_delay)

                    # if delay > max_delay:
                    #     print(delay, max_delay, delay > max_delay)

                    attack_frame = [attack_ts, attack_id, dlc, payload, 1]
                    
                    

                    ptr = ind + 1

                    out.append(attack_frame)
                    out.append(list(data.iloc[ind].values))
                    # print("ptr=ind",ind)
                    if ind == last_appended + 1:
                        last_appended = ind
                    else:
                        # print(last_appended, ind)
                        last_appended = ind
                        # print("SKIPPED LAST APPEND")
            
    # print(last_appended+1)
    out.extend(data.iloc[last_appended+1:].values.tolist())  
                    
    return out

In [62]:
##Issue is max_delay
out = attack(normal_data, 0.6)

100%|██████████| 100000/100000 [19:49<00:00, 84.10it/s]  


In [63]:
out_df = pd.DataFrame(columns = ['Timestamp','ID', 'DLC', 'Payload', 'label'], data = out)
out_df = out_df.assign(IAT=out_df['Timestamp'].diff().fillna(0))
out_df[out_df['IAT'] < 0]

,Timestamp,ID,DLC,Payload,label,IAT


In [67]:
out_df['IAT'] = out_df['IAT'] * 1e6
out_df.head()

,Timestamp,ID,DLC,Payload,label,IAT
0,1.479121e+09,848.0,8.0,3.716925e+17,0.0,0.000000
1,1.479121e+09,704.0,8.0,1.441152e+18,0.0,221.014023
2,1.479121e+09,1072.0,8.0,0.000000e+00,0.0,553.846359
3,1.479121e+09,1201.0,8.0,0.000000e+00,0.0,237.941742
4,1.479121e+09,497.0,8.0,0.000000e+00,0.0,248.193741


In [75]:
normal_data[normal_data['DLC'] != 8]['ID'].unique()

array([ 688, 1520])

In [71]:
normal_data[normal_data['DLC'] != 8]['IAT'].median() * 1e6

225.067138671875

In [72]:
normal_data[normal_data['DLC'] ==8]['IAT'].median() * 1e6

240.08750915527344

In [73]:
median_iat = normal_data.groupby('ID')['IAT'].median().reset_index()

# Sort the result by 'IAT' in ascending order
median_iat_sorted = median_iat.sort_values(by='IAT', ascending=True)

# Print the sorted IDs and median IATs
print(median_iat_sorted)


      ID       IAT
25  1520  0.000138
7    399  0.000226
9    608  0.000229
20  1201  0.000229
16   880  0.000232
14   809  0.000233
4    305  0.000234
2    161  0.000234
8    497  0.000236
19  1088  0.000237
10   672  0.000239
22  1349  0.000240
5    320  0.000241
6    339  0.000242
3    304  0.000243
21  1264  0.000246
23  1440  0.000247
26  1680  0.000248
0      2  0.000251
24  1442  0.000251
11   688  0.000275
1    160  0.000442
15   848  0.000442
17  1072  0.000467
13   790  0.000677
18  1087  0.000958
12   704  0.000986


In [76]:
median_iat_sorted

,ID,IAT
25,1520,0.000138
7,399,0.000226
9,608,0.000229
20,1201,0.000229
16,880,0.000232
14,809,0.000233
4,305,0.000234
2,161,0.000234
8,497,0.000236
19,1088,0.000237


In [65]:
out_df.iloc[10000:10050]

,Timestamp,ID,DLC,Payload,label,IAT
10000,1.479121e+09,809.0,8.0,9.779987e+18,0.0,0.000241
10001,1.479121e+09,848.0,8.0,3.717453e+17,2.0,0.000236
10002,1.479121e+09,848.0,8.0,3.716925e+17,0.0,0.000230
10003,1.479121e+09,1349.0,8.0,1.556444e+19,2.0,0.000234
10004,1.479121e+09,1349.0,8.0,1.556444e+19,0.0,0.000243
10005,1.479121e+09,1520.0,2.0,6.246400e+04,2.0,0.000242
10006,1.479121e+09,1520.0,2.0,6.246400e+04,0.0,0.000265
10007,1.479121e+09,1072.0,8.0,0.000000e+00,2.0,0.000132
10008,1.479121e+09,2.0,8.0,5.260830e+05,1.0,0.000287
10009,1.479121e+09,1072.0,8.0,0.000000e+00,0.0,0.000248


In [68]:
stats_df = out_df.groupby(['ID', 'label'])['IAT'].agg(['mean', 'median', 'std']).reset_index()

# Print the statistics in a neat and organized fashion
print("Mean, Median, and Standard Deviation of IAT for each ID and Label:")
for id_val in out_df['ID'].unique():
    print(f"\nID: {id_val}")
    for label in out_df['label'].unique():
        subset = stats_df[(stats_df['ID'] == id_val) & (stats_df['label'] == label)]
        if not subset.empty:
            mean_val = subset['mean'].values[0]
            median_val = subset['median'].values[0]
            std_val = subset['std'].values[0]
            print(f"  Label {label}: Mean = {mean_val:.3f}, Median = {median_val:.3f}, Std = {std_val:.3f}")

Mean, Median, and Standard Deviation of IAT for each ID and Label:

ID: 848.0
  Label 0.0: Mean = 275.354, Median = 234.127, Std = 210.583
  Label 2.0: Mean = 233.446, Median = 240.088, Std = 19.098
  Label 1.0: Mean = 778.579, Median = 435.829, Std = 612.620

ID: 704.0
  Label 0.0: Mean = 405.365, Median = 246.048, Std = 671.073
  Label 2.0: Mean = 235.733, Median = 240.088, Std = 16.062
  Label 1.0: Mean = 1005.301, Median = 436.068, Std = 927.356

ID: 1072.0
  Label 0.0: Mean = 291.751, Median = 247.955, Std = 163.416
  Label 2.0: Mean = 230.707, Median = 236.988, Std = 25.166
  Label 1.0: Mean = 1004.351, Median = 685.334, Std = 786.409

ID: 1201.0
  Label 0.0: Mean = 227.175, Median = 227.928, Std = 20.200
  Label 2.0: Mean = 250.101, Median = 250.101, Std = nan
  Label 1.0: Mean = 1807.332, Median = 1807.332, Std = 1907.229

ID: 497.0
  Label 0.0: Mean = 240.610, Median = 241.041, Std = 10.477
  Label 2.0: Mean = 230.184, Median = 236.034, Std = 18.397

ID: 339.0
  Label 0.0: Mea

In [53]:
normal_data.iloc[10000:1050]

,Timestamp,ID,DLC,Payload,label,IAT
1000,1.479121e+09,2,8,525175,0,0.000224
1001,1.479121e+09,339,8,1095233372160,0,0.000242
1002,1.479121e+09,790,8,370548651961024637,0,0.000237
1003,1.479121e+09,399,8,18317828534380003328,0,0.000245
1004,1.479121e+09,608,8,1955727931050260256,0,0.000224
1005,1.479121e+09,672,8,4611803790742707456,0,0.000238
1006,1.479121e+09,304,8,540433050526286819,0,0.000245
1007,1.479121e+09,305,8,36028798101029733,0,0.000237
1008,1.479121e+09,320,8,68100879,0,0.000245
1009,1.479121e+09,809,8,916902623815139348,0,0.000240


In [42]:
len(out)

15489

In [86]:
out[-1]

[[1479121434.900189, 848.0, 8.0, 3.717805056385354e+17, 0.0],
 [1479121434.900434, 1440.0, 8.0, 0.0, 0.0]]

In [113]:
print()

In [49]:
normal_data

,Timestamp,ID,DLC,Payload,label,IAT
0,1.479121e+09,848,8,371692544708313250,0,0.000000
1,1.479121e+09,704,8,1441151880758558720,0,0.000221
2,1.479121e+09,1072,8,0,0,0.000554
3,1.479121e+09,1201,8,0,0,0.000238
4,1.479121e+09,497,8,0,0,0.000248
...,...,...,...,...,...,...
95,1.479121e+09,880,8,9007199254740992,0,0.000251
96,1.479121e+09,1088,8,18374686483956762624,0,0.000245
97,1.479121e+09,704,8,1441151880758558720,0,0.004553
98,1.479121e+09,848,8,371780505638535410,0,0.000201


In [26]:
normal_data.iloc[-1]

Timestamp    1.479121e+09
ID           1.264000e+03
DLC          8.000000e+00
Payload      5.497610e+11
label        0.000000e+00
IAT          7.479191e-04
Name: 999, dtype: float64

In [139]:
out[-1]

array([1.47912144e+09, 1.26400000e+03, 8.00000000e+00, 5.49760992e+11,
       0.00000000e+00])

999

In [37]:
# for i in range(len(out)):
#     length = len(out[i])

#     if length != 5:
#         print(length, i)
#         print(out[i])

In [43]:
out_df = pd.DataFrame(columns = ['Timestamp','ID', 'DLC', 'Payload', 'label'], data = out)
out_df = out_df.assign(IAT=out_df['Timestamp'].diff().fillna(0))
out_df[out_df['IAT'] < 0]

,Timestamp,ID,DLC,Payload,label,IAT


In [44]:
out_df = out_df.assign(IAT=out_df['Timestamp'].diff().fillna(0))

In [79]:
out_df[out_df['IAT'] < 0]

,Timestamp,ID,DLC,Payload,label,IAT


In [48]:
out_df['label'].value_counts()

0.0    100000
2.0     33472
1.0     18191
Name: label, dtype: int64

In [31]:
out_df[out_df['IAT'] < 0]['label'].unique()

array([1.])

In [32]:
out_df['label'].unique()

array([0., 2., 1.])

In [95]:
len(out_df)

115948

In [101]:
out_df['label'].value_counts()

0.0    99988
1.0    18191
2.0     9976
Name: label, dtype: int64

In [ ]:
print(frame_len(hex_to_dec(339.0), 8, hex_to_dec()))

In [74]:
out_df.iloc[10010:10015]

,Timestamp,ID,DLC,Payload,label,IAT
10010,1.479121e+09,339.0,8.0,1.095233e+12,0.0,0.000254
10011,1.479121e+09,304.0,8.0,1.841043e+19,1.0,0.000054
10012,1.479121e+09,304.0,8.0,1.379903e+19,0.0,0.000228
10013,1.479121e+09,305.0,8.0,1.981584e+18,0.0,0.000233
10014,1.479121e+09,305.0,8.0,1.682517e+19,2.0,0.000240


In [88]:
out_df[out_df['label'] == 1]['IAT'].mean()

0.0009292925402196885

In [45]:
out_df.to_csv('aux_attacks_new_post.csv', index = False)

In [40]:
out_df['label'].value_counts()

0.0    100000
1.0     18152
2.0      5360
Name: label, dtype: int64

In [21]:
# out_df.to_csv('aux_attacks.csv', index = False)

In [27]:
out_df['IAT'] = out_df['IAT'] * 1e6

In [31]:
out_df[out_df['label'] == 1]["IAT"].min()

33.855438232421875

In [22]:
out_df[out_df["label"] != 0]

,Timestamp,ID,DLC,Payload,label
10003,1.479121e+09,1072.0,8.0,0.000000e+00,1.0
10009,1.479121e+09,304.0,8.0,1.841043e+19,1.0
10013,1.479121e+09,704.0,8.0,1.441152e+18,1.0
10018,1.479121e+09,790.0,8.0,3.705531e+17,1.0
10027,1.479121e+09,1264.0,8.0,5.634997e+14,1.0
...,...,...,...,...,...
123485,1.479121e+09,2.0,8.0,6.962900e+04,1.0
123498,1.479121e+09,688.0,5.0,8.546654e+11,1.0
123500,1.479121e+09,848.0,8.0,3.717805e+17,1.0
123502,1.479121e+09,704.0,8.0,1.441152e+18,1.0


In [23]:
out_df = out_df.assign(IAT=out_df['Timestamp'].diff().fillna(0))
# out_df['IAT'] = out_df['IAT'] * 1e6

In [24]:
out_df[(out_df['IAT'] < 0) & (out_df["label"] == 0)]

,Timestamp,ID,DLC,Payload,label,IAT


In [25]:
out_df[(out_df['IAT'] < 0) & (out_df["label"] == 1)].head(7)

,Timestamp,ID,DLC,Payload,label,IAT


In [124]:
def calculate_bus_load(df, bus_bandwidth_kbps):
    # Convert the bus bandwidth to bits per second
    bus_bandwidth_bps = bus_bandwidth_kbps * 1000
    
    # Calculate the duration of the session
    start_time = df['Timestamp'].min()
    end_time = df['Timestamp'].max()
    total_time_sec = end_time - start_time  # Assuming timestamps are in seconds
    
    # Calculate total data transmitted in bits using the frame_len function
    total_bits_transmitted = 0
    
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Calculating Bus Load"):
        # Convert the 'ID' and 'Payload' to hex strings and pass them along with DLC
        frame_bits = frame_len(dec_to_hex(int(row['ID'])), int(row['DLC']), dec_to_hex(int(row['Payload'])))
        total_bits_transmitted += frame_bits
    
    # Calculate the bus load as a percentage
    bus_load_percentage = (total_bits_transmitted / (total_time_sec * bus_bandwidth_bps)) * 100
    
    return bus_load_percentage

In [126]:
bus_load = calculate_bus_load(normal_data[:100_000], 500)
print(f"Bus Load: {bus_load:.2f}%")

attack_bus_load = calculate_bus_load(out_df, 500)
print(f"Attack Bus Load: {attack_bus_load:.2f}%")

Calculating Bus Load: 100%|██████████| 100000/100000 [00:12<00:00, 7791.49it/s]


Bus Load: 45.77%


Calculating Bus Load: 100%|██████████| 152074/152074 [00:15<00:00, 9615.84it/s]

Attack Bus Load: 69.62%


In [119]:
normal_data

,Timestamp,ID,DLC,Payload,label,IAT
0,1.479121e+09,848,8,371692544708313250,0,0.000000
1,1.479121e+09,704,8,1441151880758558720,0,0.000221
2,1.479121e+09,1072,8,0,0,0.000554
3,1.479121e+09,1201,8,0,0,0.000238
4,1.479121e+09,497,8,0,0,0.000248
...,...,...,...,...,...,...
988866,1.479122e+09,688,5,738819049343,0,0.001843
988867,1.479122e+09,790,8,376068213553234298,0,0.002187
988868,1.479122e+09,399,8,18316421159497433088,0,0.000223
988869,1.479122e+09,608,8,3618705183201253660,0,0.000223


In [28]:
x = 31564
out_df.iloc[x-5:x+20]

,Timestamp,ID,DLC,Payload,label,IAT
31559,1.479121e+09,1520.0,2.0,6.246400e+04,1.0,0.002212
31560,1.479121e+09,160.0,8.0,9.260188e+18,0.0,0.000143
31561,1.479121e+09,160.0,8.0,7.242470e+18,1.0,0.000196
31562,1.479121e+09,161.0,8.0,9.187625e+18,0.0,0.000206
31563,1.479121e+09,1520.0,2.0,6.246400e+04,0.0,0.000329
31564,1.479121e+09,704.0,8.0,1.441152e+18,1.0,-0.000039
31565,1.479121e+09,2.0,8.0,2.559000e+03,0.0,0.000201
31566,1.479121e+09,339.0,8.0,1.095233e+12,0.0,0.000208
31567,1.479121e+09,704.0,8.0,1.441152e+18,1.0,0.000043
31568,1.479121e+09,704.0,8.0,1.441152e+18,0.0,0.000201


In [56]:
normal_data.iloc[10:20]

,Timestamp,ID,DLC,Payload,label,IAT
10,1.479121e+09,320,8,337649894,0,251.054764
11,1.479121e+09,608,8,1955727931050323513,0,229.120255
12,1.479121e+09,672,8,6917646799956401408,0,236.034393
13,1.479121e+09,790,8,370566244147069053,0,229.835510
14,1.479121e+09,809,8,9779986690480275476,0,234.127045
15,1.479121e+09,1349,8,15564440904897921024,0,241.041183
16,1.479121e+09,688,5,609868580610,0,181.913376
17,1.479121e+09,1087,8,18121048359438336,0,657.081604
18,1.479121e+09,880,8,9007199254740992,0,250.816345
19,1.479121e+09,1088,8,18374686483956893696,0,241.041183


In [59]:
normal_data.iloc[10:100]

,Timestamp,ID,DLC,Payload,label,IAT
10,1.479121e+09,320,8,337649894,0,251.054764
11,1.479121e+09,608,8,1955727931050323513,0,229.120255
12,1.479121e+09,672,8,6917646799956401408,0,236.034393
13,1.479121e+09,790,8,370566244147069053,0,229.835510
14,1.479121e+09,809,8,9779986690480275476,0,234.127045
...,...,...,...,...,...,...
95,1.479121e+09,880,8,9007199254740992,0,251.054764
96,1.479121e+09,1088,8,18374686483956762624,0,245.094299
97,1.479121e+09,704,8,1441151880758558720,0,4552.841187
98,1.479121e+09,848,8,371780505638535410,0,200.986862


In [69]:
length = frame_len(dec_to_hex(1088), 8, dec_to_hex(18374686483956762624))
print(transmission_time(length, 512) * 1e6)

355.46875


In [53]:
0.000242 * 1e6

242.0

In [48]:
dec_to_hex(1349)

'545'

In [46]:
normal_data[normal_data["ID"] == 1349.0]

,Timestamp,ID,DLC,Payload,label,IAT
15,1.479121e+09,1349,8,15564440904897921024,0,0.000241
33,1.479121e+09,1349,8,15564440900602953728,0,0.000245
54,1.479121e+09,1349,8,15589492173530202112,0,0.000234
72,1.479121e+09,1349,8,15564440909192888320,0,0.000242
92,1.479121e+09,1349,8,15564440909192888320,0,0.000233
...,...,...,...,...,...,...
9905,1.479121e+09,1349,8,15564440926372757504,0,0.000243
9926,1.479121e+09,1349,8,15564440922077790208,0,0.000244
9946,1.479121e+09,1349,8,15564440913487855616,0,0.000241
9964,1.479121e+09,1349,8,15597936444306358272,0,0.000233


In [42]:
normal_data[normal_data["ID"] == 1349.0]['IAT'].std() * 1e6

5.554728342588226

In [37]:
x = 1011
out_df.iloc[x-5:x+20]

,Timestamp,ID,DLC,Payload,label,IAT
1006,1.479121e+09,304.0,8.0,5.404331e+17,0.0,607.252121
1007,1.479121e+09,305.0,8.0,3.602880e+16,0.0,576.972961
1008,1.479121e+09,320.0,8.0,6.810088e+07,0.0,600.814819
1009,1.479121e+09,809.0,8.0,9.169026e+17,0.0,437.974930
1010,1.479121e+09,1349.0,8.0,1.556444e+19,0.0,575.065613
1011,1.479121e+09,1440.0,8.0,0.000000e+00,1.0,-1929.759979
1012,1.479121e+09,688.0,5.0,6.098686e+11,0.0,89.883804
1013,1.479121e+09,688.0,5.0,6.098686e+11,2.0,303.983688
1014,1.479121e+09,1087.0,8.0,1.812105e+16,0.0,334.978104
1015,1.479121e+09,880.0,8.0,9.007199e+15,0.0,329.017639


In [100]:
print(out_df.iloc[10018]['Timestamp'])

1479121439.972146


In [101]:
normal_data.iloc[10009]['Timestamp']

1479121439.972146

In [88]:
normal_data['IAT'] = normal_data['IAT'] * 1e6
normal_data.iloc[9_990:10_020]

,Timestamp,ID,DLC,Payload,label,IAT
9990,1.479121e+09,320,8,68104083,0,257.968903
9991,1.479121e+09,704,8,1441151880758558720,0,3651.857376
9992,1.479121e+09,1087,8,91586017281638912,0,272.989273
9993,1.479121e+09,880,8,9007199254740992,0,244.140625
9994,1.479121e+09,1088,8,18374969058445951525,0,239.849091
9995,1.479121e+09,790,8,373359012439130237,0,1187.086105
9996,1.479121e+09,399,8,18317828534380986368,0,216.960907
9997,1.479121e+09,608,8,2823243704877341499,0,217.914581
9998,1.479121e+09,672,8,6917646799956401408,0,237.226486
9999,1.479121e+09,688,5,25837242129,0,184.774399


In [23]:
out_df[(out_df['IAT'] < 0) & (out_df['label'] == 2)]

,Timestamp,ID,DLC,Payload,label,IAT


In [118]:
slice = 10007
out_df.iloc[slice-5:slice+15]

,Timestamp,ID,DLC,Payload,label,IAT
10002,1.479121e+09,848.0,8.0,3.717805e+17,2.0,345.945358
10003,1.479121e+09,1349.0,8.0,1.556444e+19,0.0,235.080719
10004,1.479121e+09,1349.0,8.0,1.556444e+19,2.0,368.118286
10005,1.479121e+09,1520.0,2.0,6.246400e+04,0.0,374.794006
10006,1.479121e+09,497.0,8.0,0.000000e+00,2.0,260.114670
10007,1.479121e+09,1072.0,8.0,0.000000e+00,1.0,-142.812729
10008,1.479121e+09,1072.0,8.0,0.000000e+00,0.0,89.883804
10009,1.479121e+09,1072.0,8.0,0.000000e+00,2.0,89.883804
10010,1.479121e+09,1201.0,8.0,1.302122e+18,0.0,213.861465
10011,1.479121e+09,2.0,8.0,5.283630e+05,2.0,368.118286


In [67]:
out_df[out_df['label'] == 2]['IAT'].min()

89.88380432128906

In [48]:
for i in range(len(data_out)):
    if len(data_out[i]) != 5:
        print(i, len(data_out[i]))

203334 6
203335 6
203336 6
203337 6
203338 6
203339 6
203340 6
203341 6
203342 6
203343 6
203344 6
203345 6


In [50]:
len(data_out)

203346

In [45]:
normal_data.drop(['IAT'],axis=1)

,Timestamp,ID,DLC,Payload,label
0,1.479121e+09,848,8,371692544708313250,0
1,1.479121e+09,704,8,1441151880758558720,0
2,1.479121e+09,1072,8,0,0
3,1.479121e+09,1201,8,0,0
4,1.479121e+09,497,8,0,0
...,...,...,...,...,...
99995,1.479121e+09,608,8,3202177193253183055,0
99996,1.479121e+09,672,8,117772315319552,0
99997,1.479121e+09,809,8,9779705215488171028,0
99998,1.479121e+09,880,8,9851624184872960,0


In [26]:
# data_out = attack_old(normal_data, 0.9)
out_df = pd.DataFrame(data_out, columns = ['Timestamp','ID', 'DLC', 'Payload', 'label'])
out_df = out_df.assign(IAT=out_df['Timestamp'].diff().fillna(0))

out_df['IAT'] = out_df['IAT'] * 1e6

 19%|█▊        | 18637/100000 [00:19<02:08, 633.70it/s]  

DID NOT HAPPEN


 28%|██▊       | 28249/100000 [00:39<02:36, 459.81it/s]

DID NOT HAPPEN


 29%|██▉       | 28760/100000 [00:40<02:08, 552.54it/s]

...

 94%|█████████▍| 94299/100000 [03:14<00:19, 290.83it/s]

DID NOT HAPPEN


 99%|█████████▊| 98559/100000 [03:26<00:03, 360.55it/s]

DID NOT HAPPEN


100%|██████████| 100000/100000 [03:31<00:00, 473.83it/s]


In [17]:
out_df.to_csv('temp.csv')

In [16]:
!rm temp.csv

In [22]:
normal_data.iloc[0]['ID'].item()

848.0

In [33]:
arb_priorities

{2: 0,
 160: 1,
 161: 2,
 304: 3,
 305: 4,
 320: 5,
 339: 6,
 399: 7,
 497: 8,
 608: 9,
 672: 10,
 688: 11,
 704: 12,
 790: 13,
 809: 14,
 848: 15,
 880: 16,
 1072: 17,
 1087: 18,
 1088: 19,
 1201: 20,
 1264: 21,
 1349: 22,
 1440: 23,
 1442: 24,
 1520: 25,
 1680: 26}

In [38]:
rank = 7
keys_below = [key for key, value in arb_priorities.items() if value < 2]
keys_below

[2, 160]

In [41]:
lst = [0,1,2,3]
lst[:2]

[0, 1]

In [46]:
len([])

0

In [24]:
normal_data.shape[0]

100000

In [38]:
# normal_data['IAT'] = normal_data['IAT'] * 1e6
normal_data['prev_packet_tt'] = 0

# Iterate over the DataFrame rows starting from the second row
for i in range(1, len(normal_data)):
    prev_id = normal_data.loc[i-1, 'ID']
    prev_dlc = normal_data.loc[i-1, 'DLC']
    prev_payload = normal_data.loc[i-1, 'Payload']

    print(prev_id, prev_dlc, prev_payload, sep='\n')
    frame_length = frame_len(dec_to_hex(prev_id), prev_dlc, dec_to_hex(prev_payload))
    print(frame_length)
    
    
    # Convert ID and Payload to hexadecimal and calculate frame length
    frame_length = frame_len(dec_to_hex(prev_id), prev_dlc, dec_to_hex(prev_payload))
    
    # Calculate transmission time and update the current row's prev_packet_tt
    normal_data.loc[i, 'prev_packet_tt'] = (transmission_time(frame_length, 1000))

848
8
371692544708313250
1192


In [37]:
normal_data

,Timestamp,ID,DLC,Payload,label,IAT,prev_packet_tt
0,1.479121e+09,848,8,371692544708313250,0,0.000000,0
1,1.479121e+09,704,8,1441151880758558720,0,221.014023,0
2,1.479121e+09,1072,8,0,0,553.846359,0
3,1.479121e+09,1201,8,0,0,237.941742,0
4,1.479121e+09,497,8,0,0,248.193741,0
...,...,...,...,...,...,...,...
99995,1.479121e+09,608,8,3202177193253183055,0,237.941742,0
99996,1.479121e+09,672,8,117772315319552,0,236.034393,0
99997,1.479121e+09,809,8,9779705215488171028,0,230.073929,0
99998,1.479121e+09,880,8,9851624184872960,0,247.001648,0


In [26]:
normal_data[normal_data['IAT'] < normal_data['prev_packet_tt']]

,Timestamp,ID,DLC,Payload,label,IAT,prev_packet_tt
1,1.479121e+09,704,8,1441151880758558720,0,221.014023,1192.0
2,1.479121e+09,1072,8,0,0,553.846359,1276.0
6,1.479121e+09,2,8,10,0,252.008438,811.0
8,1.479121e+09,304,8,252202674911445149,0,241.041183,1271.0
9,1.479121e+09,305,8,36028797782261911,0,243.902206,1195.0
...,...,...,...,...,...,...,...
99995,1.479121e+09,608,8,3202177193253183055,0,237.941742,1271.0
99996,1.479121e+09,672,8,117772315319552,0,236.034393,1266.0
99997,1.479121e+09,809,8,9779705215488171028,0,230.073929,963.0
99998,1.479121e+09,880,8,9851624184872960,0,247.001648,1265.0


In [43]:
dec_to_hex(371692544708313250)

'52884666d0000a2'

In [39]:
print(frame_len(dec_to_hex(848), 8, dec_to_hex(371692544708313250)))

1192


In [32]:
(125/(500 * 1000)) * 1e6

250.0

In [30]:
47 + 64 + 3 + 11

125

In [26]:
normal_data.columns

Index(['Timestamp', 'ID', 'DLC', 'Payload', 'label', 'IAT'], dtype='object')